# Visual Explanations with Grad-CAM

We performed a visual explanation analysis with **14 new real-world images**. This new set of images consists of **seven non-meteors** and **seven meteors** optical observations from **BRAMON**. We then fed the images into the best-selected classifiers and used them to generate the predictions and the heat maps.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import argparse
import glob

from tensorflow.keras.preprocessing import image
from tf_explain.core.grad_cam import GradCAM
from utils.visualize import explainGradCamCompare
from utils.layers import ChannelAttention, SpatialAttention

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

parser = argparse.ArgumentParser()
args = parser.parse_args("")

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

%matplotlib inline

In [ ]:
# Load trained models
model_baseline = tf.keras.models.load_model('models/baseline/vgg_finalmodel_baseline.h5')
model_cbam = tf.keras.models.load_model('models/cbam/vgg_finalmodel_cbam.h5',
                                        custom_objects={
                                            "ChannelAttention": ChannelAttention,
                                            "SpatialAttention": SpatialAttention
                                        }
                                       )

In [ ]:
# Load the new set of images
meteor_images = glob.glob('data/visual_explanations/meteor/*.jpg')
nonmeteor_images = glob.glob('data/visual_explanations/nonmeteor/*.jpg')
all_images = nonmeteor_images + meteor_images

# Pre-processing
X_test = []
y_test = np.concatenate((np.zeros((7,), dtype=np.int64), np.ones((7,), dtype=np.int64))) # zero -> nonmeteor; one -> meteor
for filename in all_images:
    test_image = image.load_img(filename, target_size=(224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = test_image / 255
    X_test.append(test_image)

X_test = np.array(X_test).reshape((len(all_images), 224, 224, 3))
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Perform visual explanations with Grad-CAM
explainer = GradCAM()

fig, axes = plt.subplots(14, 3, figsize=(8, 32))
fig.tight_layout()
index = 0
for i, package in enumerate(zip(X_test, y_test)):
    img, label = package
    explainGradCamCompare(i, explainer, axes[index], img,
               label,
               model_baseline,
               model_baseline.predict(img[None]), 
               model_cbam,
               model_cbam.predict(img[None]), 
               class_names=['nonmeteor', 'meteor'])
    index += 1